In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tarfile
import requests
import os
import numpy as np
import pickle
import keras
import tensorflow_hub as hub
import tensorflow

In [ ]:
dataGenerator = keras.preprocessing.image.ImageDataGenerator(validation_split=0.3,
                                    samplewise_center=True, fill_mode="nearest",
                                    brightness_range=[0.9,1.1])

# Classifying Normal vs Malignant

## Convolution

In [ ]:
def finalNet2():
    m = keras.Sequential([
        keras.layers.Conv2D(64, (3, 3), activation="tanh", input_shape=(224, 224, 3)),
        keras.layers.BatchNormalization(),
        keras.layers.Activation('relu'),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),

        keras.layers.Conv2D(64, (3, 3), activation='relu'),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),

        keras.layers.Flatten(),
        keras.layers.Dense(1, kernel_initializer="he_normal",bias_initializer="he_normal"),
        keras.layers.BatchNormalization(),
        keras.layers.Activation('sigmoid')
    ])

    opt = keras.optimizers.SGD(learning_rate=.01)
    m.compile(optimizer=opt,
                        # loss='categorical_hinge',
                        # loss='categorical_crossentropy',
                        loss='binary_crossentropy',
                        metrics=['categorical_accuracy'])

    return(m)

## Resnet 50 Transfer

In [ ]:
def finalNet2():
    m = keras.Sequential([
        # hub.KerasLayer("https://tfhub.dev/google/imagenet/inception_resnet_v2/feature_vector/5", trainable=True),
        hub.KerasLayer("https://tfhub.dev/tensorflow/resnet_50/feature_vector/1", trainable=True),
        # hub.KerasLayer("https://tfhub.dev/google/imagenet/inception_v3/feature_vector/5", trainable=True),
        # keras.layers.Dense(1024, kernel_initializer="he_normal",bias_initializer="he_normal"),
        # keras.layers.BatchNormalization(),
        # keras.layers.Activation('relu'),

        #keras.layers.Dropout(0.3),

        keras.layers.Dense(1, kernel_initializer="he_normal",bias_initializer="he_normal"),
        keras.layers.BatchNormalization(),
        keras.layers.Activation('sigmoid')
    ])

    opt = keras.optimizers.SGD(learning_rate=.01)
    m.compile(optimizer=opt,
                        # loss='categorical_hinge',
                        # loss='categorical_crossentropy',
                        loss='binary_crossentropy',
                        metrics=['categorical_accuracy'])

    return(m)

## Train and Test

In [ ]:
m = finalNet2()

In [ ]:
train = dataGenerator.flow_from_directory("/content/drive/MyDrive/Lung Cancer Images", class_mode='binary',
                                    batch_size=32, subset="training",
                                    target_size=(224,224), classes=['malignant','normal'])
test = dataGenerator.flow_from_directory("/content/drive/MyDrive/Lung Cancer Images", class_mode='binary',
                                    batch_size=32, subset="validation",
                                    target_size=(224,224), classes=['malignant','normal'])

Found 685 images belonging to 2 classes.
Found 292 images belonging to 2 classes.


In [ ]:
m.fit(train, epochs=1, validation_data=test, class_weight =
          {
              0: 1.35, # normal 416
              1: 1, # malignant 561
          })

m.save('Final.h5')
m.save('/content/drive/MyDrive/Final_resnet_2_cat.h5')

22/22 [==============================] - 423s 19s/step - loss: 0.5135 - categorical_accuracy: 1.0000 - val_loss: 4.8992 - val_categorical_accuracy: 1.0000


# Classifying Malignant vs Benign

In [ ]:
def finalNet2alt():
    m = keras.Sequential([
        keras.layers.Conv2D(64, (3, 3), activation="tanh", input_shape=(224, 224, 3)),
        keras.layers.BatchNormalization(),
        keras.layers.Activation('relu'),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),

        keras.layers.Conv2D(64, (3, 3), activation='relu'),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),

        keras.layers.Flatten(),
        keras.layers.Dense(1, kernel_initializer="he_normal",bias_initializer="he_normal"),
        keras.layers.BatchNormalization(),
        keras.layers.Activation('sigmoid')
    ])

    opt = keras.optimizers.SGD(learning_rate=.01)
    m.compile(optimizer=opt,
                        # loss='categorical_hinge',
                        # loss='categorical_crossentropy',
                        loss='binary_crossentropy',
                        metrics=['categorical_accuracy'])

    return(m)

In [ ]:
m = finalNet2alt()

In [ ]:
train = dataGenerator.flow_from_directory("/content/drive/MyDrive/Lung Cancer Images", class_mode='binary',
                                    batch_size=32, subset="training",
                                    target_size=(224,224), classes=['benign','normal'])
test = dataGenerator.flow_from_directory("/content/drive/MyDrive/Lung Cancer Images", class_mode='binary',
                                    batch_size=32, subset="validation",
                                    target_size=(224,224), classes=['benign','normal'])

Found 376 images belonging to 2 classes.
Found 160 images belonging to 2 classes.


In [ ]:
m.fit(train, epochs=1, validation_data=test, class_weight =
          {
              0: 1, # normal 416
              1: 4.5, # malignant 561
          })

m.save('Final.h5')
# m.save('/content/drive/MyDrive/Final_resnet_2_cat_alt.h5')

12/12 [==============================] - 160s 13s/step - loss: 1.9642 - categorical_accuracy: 1.0000 - val_loss: 3.7182 - val_categorical_accuracy: 1.0000


# Classifying Normal vs Malignant vs Benign

## Convolution

In [ ]:
# 0.8628
def finalNet3():
    m = keras.Sequential([
        keras.layers.Conv2D(64, (2, 2), activation="tanh", input_shape=(224, 224, 3)),
        keras.layers.BatchNormalization(),
        keras.layers.Activation('relu'),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),

        keras.layers.Conv2D(64, (2, 2), activation='relu'),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),

        keras.layers.Flatten(),
        keras.layers.Dense(16),
        keras.layers.Dense(3, kernel_initializer="he_normal",bias_initializer="he_normal"),
        keras.layers.BatchNormalization(),
        keras.layers.Activation('softmax')
    ])

    #opt = keras.optimizers.SGD(learning_rate=.01)
    m.compile(optimizer='adam',
                        # loss='categorical_hinge',
                        # loss='categorical_crossentropy',
                        loss='sparse_categorical_crossentropy',
                        metrics=['accuracy'])

    return(m)

## Transfer Learning

In [ ]:
# 85.37
def finalNet3():
    m = keras.Sequential([
        # hub.KerasLayer("https://tfhub.dev/google/imagenet/inception_resnet_v2/feature_vector/5", trainable=True),
        hub.KerasLayer("https://tfhub.dev/tensorflow/resnet_50/feature_vector/1", trainable=True),
        # hub.KerasLayer("https://tfhub.dev/google/imagenet/inception_v3/feature_vector/5", trainable=True),
        # keras.layers.Dense(1024, kernel_initializer="he_normal",bias_initializer="he_normal"),
        # keras.layers.BatchNormalization(),
        # keras.layers.Activation('relu'),

        #keras.layers.Dropout(0.3),

        # keras.layers.Dense(2, kernel_initializer="he_normal",bias_initializer="he_normal"),
        # keras.layers.BatchNormalization(),
        # keras.layers.Activation('sigmoid')

        keras.layers.Dense(3, kernel_initializer="he_normal",bias_initializer="he_normal"),
        keras.layers.BatchNormalization(),
        keras.layers.Activation('softmax')
    ])

    opt = keras.optimizers.SGD(learning_rate=.005)
    m.compile(optimizer=opt,
                        # loss='categorical_hinge',
                        loss='categorical_crossentropy',
                        metrics=['categorical_accuracy'])

    return(m)

In [ ]:
# 82.53
def finalNet3():
    m = keras.Sequential([
        # hub.KerasLayer("https://tfhub.dev/google/imagenet/inception_resnet_v2/feature_vector/5", trainable=True),
        hub.KerasLayer("https://tfhub.dev/google/imagenet/inception_resnet_v2/classification/5", trainable=True),
        # hub.KerasLayer("https://tfhub.dev/google/imagenet/inception_v3/feature_vector/5", trainable=True),
        # keras.layers.Dense(1024, kernel_initializer="he_normal",bias_initializer="he_normal"),
        # keras.layers.BatchNormalization(),
        # keras.layers.Activation('relu'),


        keras.layers.Dense(3, kernel_initializer="he_normal",bias_initializer="he_normal"),
        keras.layers.BatchNormalization(),
        keras.layers.Activation('softmax')
    ])

    opt = keras.optimizers.SGD(learning_rate=.01)
    m.compile(optimizer=opt,
                        # loss='categorical_hinge',
                        loss='categorical_crossentropy',
                        metrics=['categorical_accuracy'])

    return(m)

In [ ]:
# 88.11
def finalNet3():
    m = keras.Sequential([
        # hub.KerasLayer("https://tfhub.dev/google/imagenet/inception_resnet_v2/feature_vector/5", trainable=True),
        hub.KerasLayer("https://tfhub.dev/tensorflow/efficientnet/b0/classification/1", trainable=True),
        # hub.KerasLayer("https://tfhub.dev/google/imagenet/inception_v3/feature_vector/5", trainable=True),
        # keras.layers.Dense(1024, kernel_initializer="he_normal",bias_initializer="he_normal"),
        # keras.layers.BatchNormalization(),
        # keras.layers.Activation('relu'),


        keras.layers.Dense(3, kernel_initializer="he_normal",bias_initializer="he_normal"),
        keras.layers.BatchNormalization(),
        keras.layers.Activation('softmax')
    ])

    opt = keras.optimizers.SGD(learning_rate=.005)
    m.compile(optimizer=opt,
                        # loss='categorical_hinge',
                        loss='categorical_crossentropy',
                        metrics=['categorical_accuracy'])

    return(m)

In [ ]:
# 88.11
def finalNet3():
    m = keras.Sequential([
        # hub.KerasLayer("https://tfhub.dev/google/imagenet/inception_resnet_v2/feature_vector/5", trainable=True),
        hub.KerasLayer("https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_xl/classification/2", trainable=True),
        # hub.KerasLayer("https://tfhub.dev/google/imagenet/inception_v3/feature_vector/5", trainable=True),
        # keras.layers.Dense(1024, kernel_initializer="he_normal",bias_initializer="he_normal"),
        # keras.layers.BatchNormalization(),
        # keras.layers.Activation('relu'),


        keras.layers.Dense(3, kernel_initializer="he_normal",bias_initializer="he_normal"),
        keras.layers.BatchNormalization(),
        keras.layers.Activation('softmax')
    ])

    opt = keras.optimizers.SGD(learning_rate=.005)
    m.compile(optimizer=opt,
                        # loss='categorical_hinge',
                        loss='categorical_crossentropy',
                        metrics=['categorical_accuracy'])

    return(m)

In [ ]:
m = finalNet3()


In [ ]:
train = dataGenerator.flow_from_directory("/content/drive/MyDrive/Lung Cancer Images", class_mode='categorical',
                                    batch_size=32, subset="training",
                                    target_size=(512,512), # target_size=(224,224),
                                    classes=['benign','malignant','normal'])
test = dataGenerator.flow_from_directory("/content/drive/MyDrive/Lung Cancer Images", class_mode='categorical',
                                    batch_size=32, subset="validation",
                                    target_size=(512,512), # target_size=(224,224),
                                    classes=['benign','malignant','normal'])

Found 769 images belonging to 3 classes.
Found 328 images belonging to 3 classes.


In [ ]:
m.fit(train, epochs=5, validation_data=test, class_weight =
          {
              0: 1, # normal 416
              1: 4.5, # malignant 561
              2: 3.5 # benign 120
          })

m.save('Final.h5')
m.save('/content/drive/MyDrive/Final_resnet_3_cat_env2.h5')

Epoch 1/5


ResourceExhaustedError: ignored

# Types of Malignant

In [ ]:
def finalNet4():
    m = keras.Sequential([
        hub.KerasLayer("https://tfhub.dev/tensorflow/efficientnet/b0/classification/1", trainable=True),

        keras.layers.Dense(4, kernel_initializer="he_normal",bias_initializer="he_normal"),
        keras.layers.BatchNormalization(),
        keras.layers.Activation('softmax')
    ])

    opt = keras.optimizers.SGD(learning_rate=.1)
    m.compile(optimizer=opt,
                        loss='categorical_crossentropy',
                        metrics=['categorical_accuracy'])

    return(m)

In [ ]:
# 85.37
def finalNet4():
    m = keras.Sequential([
        hub.KerasLayer("https://tfhub.dev/tensorflow/resnet_50/feature_vector/1", trainable=True),

        keras.layers.Dense(4, kernel_initializer="he_normal",bias_initializer="he_normal"),
        keras.layers.BatchNormalization(),
        keras.layers.Activation('softmax')
    ])

    opt = keras.optimizers.SGD(learning_rate=.1)
    m.compile(optimizer=opt,
                        # loss='categorical_hinge',
                        loss='categorical_crossentropy',
                        metrics=['categorical_accuracy'])

    return(m)

In [ ]:
m = finalNet4()

In [ ]:
train = dataGenerator.flow_from_directory("/content/drive/MyDrive/CT Scan", class_mode='categorical',
                                    batch_size=32, subset="training",
                                    target_size=(224,224),
                                    classes=['ac','lcc','normal','scc'])
test = dataGenerator.flow_from_directory("/content/drive/MyDrive/CT Scan", class_mode='categorical',
                                    batch_size=32, subset="validation",
                                    target_size=(224,224),
                                    classes=['ac','lcc','normal','scc'])

Found 431 images belonging to 4 classes.
Found 182 images belonging to 4 classes.


In [ ]:
m.fit(train, epochs=20, validation_data=test, class_weight =
          {
              0: 2, # normal 416
              1: 1, # malignant 561
              2: 1.5, # benign 120
              3: 1.5
          })

m.save('Final.h5')
m.save('/content/drive/MyDrive/Final_resnet_4_cat_rn.h5')

Epoch 1/20
14/14 [==============================] - 19s 508ms/step - loss: 2.2003 - categorical_accuracy: 0.3828 - val_loss: 1.7522 - val_categorical_accuracy: 0.1868
Epoch 2/20
14/14 [==============================] - 6s 381ms/step - loss: 1.7074 - categorical_accuracy: 0.5197 - val_loss: 1.6935 - val_categorical_accuracy: 0.1868
Epoch 3/20
14/14 [==============================] - 6s 395ms/step - loss: 1.5399 - categorical_accuracy: 0.5476 - val_loss: 1.6511 - val_categorical_accuracy: 0.1868
Epoch 4/20
14/14 [==============================] - 6s 383ms/step - loss: 1.4224 - categorical_accuracy: 0.6032 - val_loss: 1.6317 - val_categorical_accuracy: 0.1868
Epoch 5/20
14/14 [==============================] - 6s 383ms/step - loss: 1.2456 - categorical_accuracy: 0.6961 - val_loss: 1.5719 - val_categorical_accuracy: 0.1978
Epoch 6/20
14/14 [==============================] - 6s 380ms/step - loss: 1.0109 - categorical_accuracy: 0.7193 - val_loss: 1.5411 - val_categorical_accuracy: 0.2747
Epo